In [1]:
import sys
import numpy as np
import astropy as ap
import matplotlib.pyplot as plt
import math as m
import argparse
from astropy.cosmology import FlatLambdaCDM
from matplotlib.backends.backend_pdf import PdfPages
import kcorrect
import kcorrect.utils as ut
import os
import pandas as pd

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("filein",help="File containing magnitudes and redshifts")
parser.add_argument("-mbin","--mbin",type=int,help="Number of Absolute Magnitude bins. Default=19",default=18)
parser.add_argument("-zmin","--zmin",type=float,help="Minimum redshift to consider in luminosity function, default=0.01", default=0.01)
parser.add_argument("-zmax","--zmax",type=float,help="Maximum redshift to consider in luminosity function, default=2", default=0.2)
parser.add_argument("-mmin","--Mmin",type=float,help="Minumum absolute magnitude to consider in luminosity function, default=-24", default=-24)
parser.add_argument("-mmax","--Mmax",type=float,help="Maximum absolute magnitude to consider in luminosity function, default=-15", default=-15)
parser.add_argument("-ama","--appmax",type=float,help='Maximum apparent magnitude to consider part of the survey COSMOS i<22.5',default=22.5)
parser.add_argument("-ami","--appmin",type=float,help='Minimum apparent magnitude to consider part of the survey COSMOS i>15',default=15)
parser.add_argument("-om","--OmegaMatter",type=float,help="Omega Matter, if you want to define your own cosmology", default=0.3)
parser.add_argument("-ho","--HubbleConstant",type=float,help="Hubble Constant if you want to define your own cosmology",default=70)
parser.add_argument("-fo","--fileout",help="Filename of PDF you want to generate",default='_LF_OUTPUT_VAL.txt')
parser.add_argument("-LCBG","--LCBGLIST",action="store_true",help="Make Luminosity Function with LCBGs only?")
parser.add_argument("-nv","--novega",action="store_true",help="Do not apply correction to switch from AB to Vega magnitudes")
args=parser.parse_args()

usage: ipykernel_launcher.py [-h] [-mbin MBIN] [-zmin ZMIN] [-zmax ZMAX]
                             [-mmin MMIN] [-mmax MMAX] [-ama APPMAX]
                             [-ami APPMIN] [-om OMEGAMATTER]
                             [-ho HUBBLECONSTANT] [-fo FILEOUT] [-LCBG] [-nv]
                             filein
ipykernel_launcher.py: error: the following arguments are required: filein


SystemExit: 2

/Users/lucashunt/programs/anaconda3/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
args=parser.parse_args('/Users/lucashunt/ASTRODATA/LCBG_LUMINOSITY_FUNCTION/COSMOS_CATALOGS/Photometry/COSMOS_CONVERTED_CATALOG.csv'.split())

In [4]:
cosmo=FlatLambdaCDM(H0=args.HubbleConstant,Om0=args.OmegaMatter)

In [5]:
print('********READING FILE********')
CATALOG=pd.read_csv(args.filein)
kcordir=os.environ["KCORRECT_DIR"]


********READING FILE********


In [37]:
print('********CALCULATING WEIGHTS********')
Magnitude_Loop_Array=np.stack((np.linspace(args.appmin,args.appmax,16)[0:15],np.linspace(args.appmin,args.appmax,16)[1:16]),axis=-1)
tf=open('weights.txt','w')
CATALOG['Spec_Weight']=np.nan
CATALOG['Color_Weight']=np.nan
CATALOG['Surface_Brightness_Weight']=np.nan
for magrange in Magnitude_Loop_Array:
    Num_Good_Spec=len(CATALOG[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0)&((CATALOG.Z_USE==1)|(CATALOG.Z_USE==2))])
    Num_Bad_Spec=len(CATALOG[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0)&((CATALOG.Z_USE==3)|(CATALOG.Z_USE==4))])
    Num_Good_Color=len(CATALOG[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0)&((CATALOG.mag_subaru_B<100)&(CATALOG.mag_subaru_V<100))])
    Num_Bad_Color=len(CATALOG[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0)&((CATALOG.mag_subaru_B>100)|(CATALOG.mag_subaru_V>100))])
    Num_Bad_Rh=len(CATALOG[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0)&(np.isnan(CATALOG.Rh))])
    Num_Good_Rh=len(CATALOG[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0)&(~np.isnan(CATALOG.Rh))])
    CATALOG.loc[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0),'Spec_Weight']=float(Num_Good_Spec+Num_Bad_Spec)/float(Num_Good_Spec)
    CATALOG.loc[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0),'Color_Weight']=float(Num_Good_Color+Num_Bad_Color)/float(Num_Good_Color)
    CATALOG.loc[(CATALOG.mag_subaru_i>magrange[0])&(CATALOG.mag_subaru_i<magrange[1])&(CATALOG.SG_MASTER==0),'Surface_Brightness_Weight']=float(Num_Good_Rh+Num_Bad_Rh)/float(Num_Good_Rh)
    

********CALCULATING WEIGHTS********


,Unnamed: 0,CATAID,COSMOS_ID_06,COSMOS_ID_08,COSMOS_ID_13,I_MAG_AUTO,RA,DEC,RA_06,DEC_06,...,k_subz-B,rest_frame_B-V,Abs_B_Mag,Surface_Brightness_B,R_eff_arcsec_F814W,R_eff_arcsec_B,is_LCBG,Spec_Weight,Color_Weight,Surface_Brightness_Weight
13987,28654,6374134,2654564,1649590.0,311052.0,18.283464,150.368238,2.708762,150.36820,2.708852,...,0.016569,0.550146,-18.716108,21.235698,1.13772,1.203910,0.0,NaN,NaN,NaN
29103,46833,6330529,2325443,1446299.0,277179.0,18.459217,150.129078,2.595622,150.12906,2.595643,...,0.316182,0.887519,-21.626677,20.044539,0.81885,0.846199,0.0,NaN,NaN,NaN
35346,54448,6319341,2252137,1401798.0,268716.0,18.159937,150.467856,2.565892,150.46784,2.565895,...,0.235904,0.737799,-21.815012,21.072366,1.44120,1.489685,0.0,NaN,NaN,NaN
122278,158058,6034642,2358886,1471063.0,279941.0,18.472322,149.937991,2.606234,149.93797,2.606254,...,0.332137,0.901622,-21.534621,20.176060,0.84132,0.869764,0.0,NaN,NaN,NaN
128804,164658,6027429,2263485,1409914.0,255708.0,18.421250,150.396829,2.519116,150.39681,2.519129,...,-0.110859,0.347834,-21.068270,20.764960,1.21854,1.273196,1.0,NaN,NaN,NaN
129015,164875,6027161,1671517,1039606.0,-9999.0,21.205343,149.990042,2.156277,149.99007,2.156302,...,-0.229897,0.398727,-22.068442,17.938273,0.25557,0.255612,1.0,NaN,NaN,NaN
129421,165289,6025915,2827617,1755009.0,298282.0,22.009539,149.669318,2.661774,149.66930,2.661791,...,-0.365280,0.294110,-24.255273,17.075572,0.49326,0.496875,1.0,NaN,NaN,NaN
129671,165539,6025552,2773826,1722932.0,311690.0,18.371317,149.844828,2.712551,149.84481,2.712589,...,0.134592,0.915731,-20.870829,20.632335,0.92403,0.962040,0.0,NaN,NaN,NaN
130280,166152,6024469,2631988,1635785.0,289974.0,18.478516,150.547302,2.635529,150.54728,2.635575,...,0.025495,0.612731,-19.980716,21.208190,1.10442,1.159132,0.0,NaN,NaN,NaN
130322,166195,6024426,2623969,1630678.0,298697.0,18.535265,150.458546,2.668679,150.45853,2.668700,...,0.237394,0.967952,-20.083549,22.716760,1.91004,1.995924,0.0,NaN,NaN,NaN


In [ ]:
LUMFUNC_CATALOG=LCBG_CATALOG[(CATALOG.Z_USE<3)&(CATALOG.mag_subaru_i<args.appmax)&(CATALOG.mag_subaru_i>args.appmin)&(CATALOG.SG_MASTER==0)&(CATALOG.Z_BEST>=args.zmin)&(CATALOG.Z_BEST>=args.zmax)]